# Retrieval API - Step 1 - Get Chunks

## Code Initialization
Dependencies and environment initialization.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `.env.example` file as template.

In [27]:
import os
import requests as r
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [28]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
USER_EMAIL = 'retrieval.api@mycompany.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [29]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token created for user {USERNAME.split('@')[0].split('-')[0]}"))
else:
    display(Markdown(f"**Authentication Failed**: Cannot obtain the Bearer token for the user {USERNAME.split('@')[0].split('-')[0]}"))

**Authentication Successful**: Bearer token created for user 9ZZZ159100

This creates the headers to be used in the Retrieval API requests.

Additionally, `user_email_hash` is calculated as `user_id`. This must be a unique and constant value per user, so the API statistics keep track of the right number of users. The recommendation is calculating a MD5 hash of the user email. This value is always the same and do not expose any PII.

In the case of `work_id`, this value must be calculated differently for each API request. For convenience, it can be taken from the environment's transaction if it exists, or calculated randomly before calling the API. In this case it is calculated randomly before the API request. See the payload section below.

In [30]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

user_email_hash = u.md5_hash(USER_EMAIL)

## Factiva Retrieval API Query

### Prompt
Add the prompt to be sent to the Retrieval API

In [31]:
# "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
# "中国2025年计划进行哪些月球探测任务？其主要目标是什么？"   # What are China's planned missions to the Moon in 2025, and what are their primary objectives?
# "今後12か月間の日本の半導体業界の見通しはどのようなものですか?"  # What are the perspectives for the semiconductors industry in Japan for the next 12 months?
# "Summarise the latest earnings report from Microsoft Corp"
# "What are the perspectives of bitcoin for the next 12 months?"
frapi_prompt = "Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide"

### Search Filters
* **Language**: Factiva language code(s), composed of two characters. Please see [Taxonomy - Languages](9_taxonomy_languages.ipynb)
* **Organization**: Factiva company/orgnization code(s). Please see [Taxonomy - Organizations](9_taxonomy_organizations.ipynb)
* **News Subjects**: Factiva company/orgnization code(s). Please see [Taxonomy - News Subjects](9_taxonomy_news_subjects.ipynb)
* **Industries**: Factiva Industries. Please see [Taxonomy - Industries](9_taxonomy_industries.ipynb)
* **Sources**: Factiva Sources (publishers). Please see [Taxonomy - Sources](9_taxonomy_sources.ipynb)
* **Regions**: Factiva Regions. Please see [Taxonomy - Regions](9_taxonomy_regions.ipynb)

### Date: Predefined and Custom Ranges
* **Predefined Ranges (`days_range`)**: Can be a value like: `LastDay`, `Last2Days`, `LastWeek`, `Last2Weeks`, `LastMonth`, `Last2Months`, `Last3Months`, `Last6Months`, `LastYear`, `Last2Years`, `Last5Years`, `AllDates`.
* **Custom Ranges (`custom`)**: Requires a `from` and `to` dates in ISO format (`YYYY-MM-DD`).

### API Payload
Assemble the Retrieval API payload

In [32]:
work_id = u.generate_random_guid()

frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 10,    # max 50
      "query": {
        "value": frapi_prompt,
        "search_filters": [
          { "scope": "Language", "value": "en or es or it or zhcn or zhtw or de or fr" },
          # { "scope": "Organization", "value": "TESLMI" },
          # { "scope": "NewsSubject", "value": "C151" },
          { "scope": "Industry", "value": "IAUT" },
          # { "scope": "Source", "value": "DJDN or J or WSJO or B" },
          # { "scope": "Region", "value": "USA" }
        ],
        "date":{
          # "custom": {
          #   "from": "2025-04-01",
          #   "to": "2025-05-31"
          # }
        "days_range": "Last3Months"
        }
      },
      "metrics_data": {
        "user_id": user_email_hash,
        "work_id": work_id,
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}

## Send Retrieval API Request and Receive Chunks Back

In [33]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print & Save Chunks

In [34]:
chunks_list = chunks_resp.json()['data']

u.save_dict_json(chunks_list, f"chunks.json")
u.save_dict_json(frapi_query, f"query.json")

# u.print_full_chunks(chunks_list)
u.print_partial_chunks(chunks_list)

### Americans Still Hit the Car Lots in May -- WSJ

**Dow Jones Newswires** - 2025-06-03 - DJDN000020250603el63002rw - Lang: en

Americans are still buying a lot of cars, despite broader consumer concerns about the economy and rising prices as a result of tariffs. Americans Stil...

---

### 特朗普计划上调钢铝关税的举动导致汽车股普遍走低

...

---

### Consultations Sought with US on Auto Parts Levies

**The Economic Times - Delhi Edition** - 2025-06-04 - ECTDEL0020250604el640000o - Lang: en

SAFEGUARD AGREEMENTParis: Days after India reserved the right to impose retaliatory duties on select US products in response to American tariffs on st...

---

## Retrieval API Conclusion
Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Next Step
Use any of the notebooks named as `2_llm_answer_*.ipynb` to explore what the LLM request looks like either using only the chunks resopnse or a combination of other data and the provided article chunks.